In [ ]:
#adicionar 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.metrics import accuracy_score

#PR 2a

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import os

cwd = os.getcwd() 
files = os.listdir(cwd)
print("Files in %r: %s" % (cwd, files))

In [ ]:
data=pd.read_csv('HR_DS.csv')

In [ ]:
pd.set_option('display.max_columns', 40)
data.head(5)

Definir perguntas para responder
Story 

Data Preparation/Clean

In [ ]:
data.describe()
#Checking the Null Values
pd.set_option('display.max_rows', 200)
#data.isnull().sum() #no null values
#data.info()

#check for  duplicate records and have the unique records to be included into your dataset:
#data[data.duplicated()] #zero duplicated

In [ ]:
#Over18 : ['Y']
#Y    1470
#so tem um unico valor, por isso vou apagar por ser inutil
#data['StandardHours'].unique() tambem so tem 1 valor, que é sempre 80
#data['EmployeeCount'].unique() tambem so tem 1 valor, que é 1

#não trazem valor 

data=data.drop('Over18', axis=1)
data=data.drop('StandardHours', axis=1)
data=data.drop('EmployeeCount', axis=1)

In [ ]:
#ver os data types e ver os seus valores unicos

for column in data.columns:
    if data[column].dtype==object:
        print(str(column) + ' : ' + str(data[column].unique()))
        print(data[column].value_counts())
        print('_______________________________________________________________')


In [ ]:
#transformar as colunas que não sao numericas, em numericas pois os modelos n aceitam colunas texto
from sklearn.preprocessing import LabelEncoder

for column in data.columns:
    if data[column].dtype == np.int64:
            continue
    data[column]=LabelEncoder().fit_transform(data[column])

In [ ]:
turnover_Summary = data.groupby('Attrition')
turnover_Summary.mean()

Analise descritiva dos dados-Gráficos


In [ ]:
#Analise das varivaies categorcias vs attrition
plt.subplots(figsize=(5,5))
data['Attrition'].value_counts(normalize = True)
data['Attrition'].value_counts(dropna = False).plot.bar(color=['green','red'])
plt.show()


In [ ]:
plt.subplots(figsize=(12,4))
sns.countplot(x='Age', hue='Attrition', data=data, palette='colorblind')

#malta dos 35/36 anos está toda empregada
#29 e 31 a malta está a sair

In [ ]:
sns.countplot(hue=data.Attrition, x=data.BusinessTravel)
plt.show()

In [ ]:
#Departamento na razão de saida
sns.countplot(hue=data.Attrition, x=data.Department)
plt.show()

In [ ]:
sns.countplot(hue=data.Attrition, x=data.EducationField)
plt.show()

In [ ]:
sns.countplot(hue=data.Attrition, x=data.MaritalStatus)
plt.show()

In [ ]:
sns.countplot(hue=data.Attrition, x=data.OverTime)
plt.show()

In [ ]:
sns.countplot(hue=data.Attrition, x=data.Gender)
plt.show()

In [ ]:
sns.countplot(x=data.Attrition, hue=data.EducationField)
plt.show()

In [ ]:
plt.figure(figsize=(20,10), facecolor='white')
sns.countplot(x='JobRole', hue='Attrition', data=data)

In [ ]:
sns.histplot(hue=data.Attrition, x=data.Age)
plt.show()

In [ ]:
sns.histplot(hue=data.Attrition, x=data.DistanceFromHome)
plt.show()

In [ ]:
sns.histplot(x=data.MonthlyIncome, hue=data.Attrition)
plt.show()

Modelo Preditivo/

In [ ]:
# ver a correlação dos dados
plt.figure(figsize=(14,14))
sns.heatmap(data.corr(), annot=True, fmt='0.0%')

Clustering/Unsupervising 

# ML

In [ ]:
#Transform non-numeric columns into numerical columns
from sklearn.preprocessing import LabelEncoder

for column in df.columns:
        if df[column].dtype == np.number:
            continue
        df[column] = LabelEncoder().fit_transform(df[column])

C:\Users\lanur\AppData\Local\Temp\ipykernel_30072\360305055.py:5: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:


In [ ]:
#Create a new column at the end of the dataframe that contains the same value 
df['Age_Years'] = df['Age']
#Remove the first column called age 
df = df.drop('Age', axis = 1)
#Show the dataframe
df

,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,MaritalStatus2,Age_Years
0,1,2,624,2,0,1,1,1,0,64,...,0,8,0,0,6,4,0,5,1,23
1,0,1,113,1,7,0,1,2,1,31,...,1,10,3,2,10,7,1,7,0,31
2,1,2,805,1,1,1,4,3,1,62,...,0,7,3,2,0,0,0,0,1,19
3,0,1,820,1,2,3,1,3,0,26,...,0,8,3,2,8,7,3,0,0,15
4,0,2,312,1,1,0,3,0,1,10,...,1,6,3,2,2,2,2,2,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0,1,494,1,22,1,3,2,1,11,...,1,17,3,2,5,2,0,3,0,18
1466,0,2,327,1,5,0,3,3,1,12,...,1,9,5,2,7,7,1,7,0,21
1467,0,2,39,1,3,2,1,1,1,57,...,1,6,0,2,6,2,0,3,0,9
1468,0,1,579,2,1,2,3,3,1,33,...,0,17,3,1,9,6,0,8,0,31


## Split de data

In [ ]:
#Split the data into independent 'X' and dependent 'Y' variables
X = df.iloc[:, 1:df.shape[1]].values 
Y = df.iloc[:, 0].values 

## dataset para treino

In [ ]:
# Split the dataset into 75% Training set and 25% Testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [ ]:
#Use Random Forest Classification algorithm
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
forest.fit(X_train, Y_train)

## Accuracy on the training data

In [ ]:
#Get the accuracy on the training data
forest.score(X_train, Y_train)

In [ ]:
#Show the confusion matrix and accuracy for  the model on the test data
#Classification accuracy is the ratio of correct predictions to total predictions made.
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, forest.predict(X_test))
  
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]
  
print(cm)
print('Model Testing Accuracy = "{}!"'.format(  (TP + TN) / (TP + TN + FN + FP)))
print()# Print a new line

In [ ]:
# Return the feature importances (the higher, the more important the feature).
importances = pd.DataFrame({'feature':df.iloc[:, 1:df.shape[1]].columns,'importance':np.round(forest.feature_importances_,3)}) #Note: The target column is at position 0
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances

In [ ]:
#Visualize the importance
importances.plot.bar()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [ ]:
lr=LogisticRegression(C = 0.1, random_state = 42, solver = 'liblinear')
dt=DecisionTreeClassifier()
rm=RandomForestClassifier()
gnb=GaussianNB()
knn = KNeighborsClassifier(n_neighbors=3)
svm = svm.SVC(kernel='linear')

In [ ]:
for a,b in zip([lr,dt,knn,svm,rm,gnb],["Logistic Regression","Decision Tree","KNN","SVM","Random Forest","Naive Bayes"]):
    a.fit(X_train,Y_train)
    prediction=a.predict(X_train)
    y_pred=a.predict(X_test)
    score1=accuracy_score(Y_train,prediction)
    score=accuracy_score(Y_test,y_pred)
    msg1="[%s] training data accuracy is : %f" % (b,score1)
    msg2="[%s] test data accuracy is : %f" % (b,score)
    print(msg1)
    print(msg2)

In [ ]:
model_scores={'Logistic Regression':lr.score(X_test,Y_test),
             'KNN classifier':knn.score(X_test,Y_test),
             'Support Vector Machine':svm.score(X_test,Y_test),
             'Random forest':rm.score(X_test,Y_test),
              'Decision tree':dt.score(X_test,Y_test),
              'Naive Bayes':gnb.score(X_test,Y_test)
             }
model_scores

In [ ]:
from sklearn.metrics import classification_report

rm_y_preds = rm.predict(X_test)

print(classification_report(Y_test,rm_y_preds))

In [ ]:
from sklearn.metrics import classification_report

lr_y_preds = lr.predict(X_test)

print(classification_report(Y_test,lr_y_preds))

In [ ]:
model_compare=pd.DataFrame(model_scores,index=['accuracy'])
model_compare

## Feature Importance

### Logistic Regression

In [ ]:
feature_dict=dict(zip(df.columns,list(lr.coef_[0])))
feature_dict

In [ ]:
feature_df=pd.DataFrame(feature_dict,index=[0])
feature_df.T.plot(kind="bar",legend=False,title="Feature Importance")